In [1]:
from csromer.simulation import FaradayThinSource,FaradayThickSource
from csromer.reconstruction import Parameter
import numpy as np
from IPython.display import clear_output
from csromer.io import Reader, Writer
from csromer.base import Dataset
from csromer.transformers import DFT1D, NUFFT1D
import matplotlib.pyplot as plt
from csromer.objectivefunction import OFunction
from csromer.utils import real_to_complex, complex_to_real
from csromer.objectivefunction import TSV, TV, L1, Chi2
from csromer.optimization import FISTA, ADMM, SDMM, GradientBasedMethod
from csromer.utils import Gaussian
from csromer.dictionaries.discrete import DiscreteWavelet
from csromer.dictionaries.undecimated import UndecimatedWavelet
from csromer.transformers import Gridding, MeanFlagger, HampelFlagger
from csromer.faraday_sky import FaradaySky
from scipy.constants import c
from pynufft import NUFFT
from scipy import signal as sci_signal
import itertools
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as un
from astropy.stats import sigma_clipped_stats
import pandas as pd

In [2]:
# Read cubes
Q_name = "/home/miguel/Documents/datasets/A1314_JVLA/cubes/A1314_Q_corrected.fits"
U_name = "/home/miguel/Documents/datasets/A1314_JVLA/cubes/A1314_U_corrected.fits"

hdul_q = fits.open(Q_name, memmap=True)
hdul_u = fits.open(U_name, memmap=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/miguel/Documents/datasets/A1314_JVLA/cubes/A1314_Q_corrected.fits'

In [ ]:
# Read spectral index
spc_idx_name = "/home/miguel/Documents/datasets/A1314_JVLA/cubes/alpha_0.fits"
hdul_alpha = fits.open(spc_idx_name, memmap=True)

In [ ]:
# Read frequencies
freq_name = "/home/miguel/Documents/datasets/A1314_JVLA/cubes/freqs.npy"
freqs = np.load(freq_name)

In [ ]:
# Get LOS
#coord = SkyCoord("11:33:59.1755+49:03:44.108", unit=(un.hourangle, un.deg), frame="fk5")
coord = SkyCoord(ra=173.498*un.deg, dec=49.063*un.deg, frame="fk5")
#coord = SkyCoord(ra=173.803*un.deg, dec=48.966*un.deg, frame="fk5")

In [ ]:
# Get WCS
wcs = WCS(hdul_q[0].header)

In [ ]:
# Get pixel
x,y = coord.to_pixel(wcs, origin=0)
x = int(x)
y = int(y)

In [ ]:
q_data = np.flipud(hdul_q[0].data)
u_data = np.flipud(hdul_u[0].data)

In [ ]:
# Get sigma
mean_q, median_q, std_q = sigma_clipped_stats(q_data[:, 0:300, 0:300], sigma=3, axis=(1,2))
mean_u, median_u, std_u = sigma_clipped_stats(u_data[:, 0:300, 0:300], sigma=3, axis=(1,2))
#std_q = np.nanstd(q_data[:, 0:100, 0:100], axis=(1, 2))
#std_u = np.nanstd(u_data[:, 0:100, 0:100], axis=(1, 2))

In [ ]:
sigma_qu = 0.5*(std_q + std_u)

In [ ]:
data = q_data[:,y,x] + 1j * u_data[:,y,x]
alpha = hdul_alpha[0].data[y,x]

In [ ]:
measurements = Dataset(nu=freqs, data=data, sigma=sigma_qu, spectral_idx=alpha)

In [ ]:
# Get galactic RM contribution
f_sky = FaradaySky(filename="./../faradaysky/faraday2020v2.hdf5")
gal_mean, gal_std = f_sky.galactic_rm(coord.ra, coord.dec, frame="fk5")

In [ ]:
measurements.subtract_galacticrm(gal_mean.value)

In [ ]:
plt.plot(measurements.lambda2, measurements.data.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, measurements.data.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(measurements.data), 'g.', label=r"$|P|$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'Jy/beam')
#plt.legend(loc='upper right')
plt.tight_layout()
#plt.savefig('polarized.png', dpi=100, bbox_inches='tight')

In [ ]:
plt.plot(measurements.lambda2, measurements.sigma, 'k.', label=r"$\sigma$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'1/(Jy/beam)$^2$')

In [ ]:
## Flagging dataset ##
normal_flagger = MeanFlagger(data=measurements, nsigma=5.0, delete_channels=True)
idxs, outliers_idxs = normal_flagger.run()

#hampel_flagger = HampelFlagger(data=measurements, nsigma=5.0, delete_channels=True)
#idxs, outliers_idxs = hampel_flagger.run()

In [ ]:
plt.plot(measurements.lambda2, measurements.data.real, 'k.', label=r"Stokes $Q$")
plt.plot(measurements.lambda2, measurements.data.imag, 'c.', label=r"Stokes $U$")
plt.plot(measurements.lambda2, np.abs(measurements.data), 'g.', label=r"$|P|$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'Jy/beam')
#plt.legend(loc='upper right')
plt.tight_layout()
#plt.savefig('polarized.png', dpi=100, bbox_inches='tight')

In [ ]:
plt.plot(measurements.lambda2, measurements.sigma, 'k.', label=r"$\sigma$")
plt.xlabel(r'$\lambda^2$[m$^{2}$]')
plt.ylabel(r'Jy/beam')

In [ ]:
parameter = Parameter()

In [ ]:
parameter.calculate_cellsize(dataset=measurements, oversampling=8)

In [ ]:
nufft = NUFFT1D(dataset=measurements, parameter=parameter, solve=True)

In [ ]:
dft = DFT1D(dataset=measurements, parameter=parameter)

In [ ]:
F_dirty = dft.backward(measurements.data)

In [ ]:
plt.plot(parameter.phi, F_dirty.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, F_dirty.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(F_dirty), 'k-', label=r"|P|")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.xlim([-1000,1000])
plt.tight_layout()
plt.savefig('fdirty.png', dpi=100, bbox_inches='tight')

In [ ]:
noise = 0.25*0.5 * (np.std(F_dirty.real[np.abs(parameter.phi)>parameter.max_faraday_depth/1.5]) + np.std(F_dirty.imag[np.abs(parameter.phi)>parameter.max_faraday_depth/1.5]))
#noise = 1/np.sqrt(np.sum(measurements.w))
noise

In [ ]:
parameter.data = F_dirty

In [ ]:
parameter.complex_data_to_real()

In [ ]:
lambda_l1 = np.sqrt(2 * len(measurements.data) + np.sqrt(4 * len(measurements.data))) * noise
#lambda_l1 = np.sqrt(len(measurements.data)) * noise_F
#lambda_l1 /= 10
#lambda_l1=0.5
lambda_tsv = 0.0
chi2 = Chi2(dft_obj=nufft)
l1 = L1(reg=lambda_l1)
tsv = TSV(reg=lambda_tsv)
# F_func = [chi2(P, dft, W), L1(lambda_l1)]
F_func = [chi2, l1, tsv]
f_func = [chi2]
g_func = [l1, tsv]

F_obj = OFunction(F_func)
f_obj = OFunction(f_func)
g_obj = OFunction(g_func)

In [ ]:
opt = FISTA(guess_param=parameter, F_obj=F_obj, fx=chi2, gx=g_obj, noise=noise, verbose=True)
#opt = FISTA(i_guess=F_real, F_obj=F_obj, fx=chi2, gx=g_obj, maxiter=1000, noise=1e-5, verbose=True)
obj, X = opt.run()

In [ ]:
X.real_data_to_complex()

In [ ]:
plt.plot(parameter.phi, X.data.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, X.data.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(X.data), 'k-', label=r"$|P|$")
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()
plt.xlim([-1000,1000])

In [ ]:
F_residual = dft.backward(measurements.residual)
plt.plot(parameter.phi, F_residual.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, F_residual.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(F_residual), 'k-', label=r"$|P|$")
plt.xlim([-1000,1000])

In [ ]:
conv_model = X.convolve(normalized=True) + F_residual

In [ ]:
plt.plot(parameter.phi, conv_model.real, 'c--', label=r"Stokes $Q$")
plt.plot(parameter.phi, conv_model.imag, 'c:', label=r"Stokes $U$")
plt.plot(parameter.phi, np.abs(conv_model), 'k-', label=r"$|P|$")
plt.xlim([-1000,1000])
#plt.axhline (y = noise_weights, color = 'b')
#plt.axhline (y = -noise_weights, color = 'b')
plt.xlabel(r'$\phi$[rad m$^{-2}$]')
plt.ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
plt.legend(loc='upper right')
plt.tight_layout()

In [ ]:
phi_idx = np.argmax(np.abs(conv_model))
parameter.phi[phi_idx]

In [ ]:
np.max(np.abs(conv_model))